<a href="https://colab.research.google.com/github/cmaguire001/Brazilian_Ecommerce_Analysis/blob/main/Brazilian_Ecommerce_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!unzip -o /content/drive/MyDrive/archive.zip -d /content/data

Archive:  /content/drive/MyDrive/archive.zip
  inflating: /content/data/olist_customers_dataset.csv  
  inflating: /content/data/olist_geolocation_dataset.csv  
  inflating: /content/data/olist_order_items_dataset.csv  
  inflating: /content/data/olist_order_payments_dataset.csv  
  inflating: /content/data/olist_order_reviews_dataset.csv  
  inflating: /content/data/olist_orders_dataset.csv  
  inflating: /content/data/olist_products_dataset.csv  
  inflating: /content/data/olist_sellers_dataset.csv  
  inflating: /content/data/product_category_name_translation.csv  


In [22]:
import pandas as pd
import sqlite3

# Load CSVs
orders = pd.read_csv('/content/data/olist_orders_dataset.csv')
customers = pd.read_csv('/content/data/olist_customers_dataset.csv')
payments = pd.read_csv('/content/data/olist_order_payments_dataset.csv')
order_items = pd.read_csv('/content/data/olist_order_items_dataset.csv')
products = pd.read_csv('/content/data/olist_products_dataset.csv')
reviews= pd.read_csv  ('/content/data/olist_order_reviews_dataset.csv')
# Create database
conn = sqlite3.connect(':memory:')

# Create SQL tables
orders.to_sql('orders', conn, index=False)
customers.to_sql('customers', conn, index=False)
payments.to_sql('payments', conn, index=False)
order_items.to_sql('order_items', conn, index=False)
products.to_sql('products', conn, index=False)
reviews.to_sql('reviews', conn, index=False)

print("✅ Setup complete!")

✅ Setup complete!


In [8]:
# ===== SECTION 2: Start of ANALYSIS

# Question 1: Total revenue
query = """
SELECT sum(payment_value) as 'Total revenue'
from payments;

"""
result = pd.read_sql(query, conn)
print(result)

   Total revenue
0   1.600887e+07


In [ ]:
# questions 2:how many orders placed each month

query= """
Select strftime('%Y-%m', order_purchase_timestamp) as 'order month'
, count(order_id) as 'orders per month'
From Orders
Group by strftime('%Y-%m', order_purchase_timestamp);


"""
result = pd.read_sql(query, conn)
print(result)

   order month  orders per month
0      2016-09                 4
1      2016-10               324
2      2016-12                 1
3      2017-01               800
4      2017-02              1780
5      2017-03              2682
6      2017-04              2404
7      2017-05              3700
8      2017-06              3245
9      2017-07              4026
10     2017-08              4331
11     2017-09              4285
12     2017-10              4631
13     2017-11              7544
14     2017-12              5673
15     2018-01              7269
16     2018-02              6728
17     2018-03              7211
18     2018-04              6939
19     2018-05              6873
20     2018-06              6167
21     2018-07              6292
22     2018-08              6512
23     2018-09                16
24     2018-10                 4


In [59]:
# Q3 finding the average order value
query="""
Select sum(payment_value)/ count (distinct order_id) as 'AOV'
From Payments;

"""
result = pd.read_sql(query, conn)
print(result)

          AOV
0  160.990267


In [ ]:
# Q4 what are the top five months for revenue?

query="""
Select strftime('%Y-%m', order_purchase_timestamp) as 'order month',
sum(payment_value) as 'total revenue'
From Orders
join payments on orders.order_id=payments.order_id
group  by strftime('%Y-%m', order_purchase_timestamp)
order by sum(payment_value) desc
limit 5;

"""

result = pd.read_sql(query, conn)
print(result)

  order month  total revenue
0     2017-11     1194882.80
1     2018-04     1160785.48
2     2018-03     1159652.12
3     2018-05     1153982.15
4     2018-01     1115004.18


In [ ]:
# Q5 How many unique customers made purchases?
query="""
select count( distinct customer_unique_id) as 'unique customers'
from customers

"""

result = pd.read_sql(query, conn)
print(result)

   unique customers
0             96096


In [ ]:
#Q6 how many orders per customer?

query="""
Select count(order_status)*1.0/count(distinct customer_unique_id) as 'OpC',
count(order_status),
count(distinct customer_unique_id)
From customers
Join orders on
Customers.customer_id=orders.customer_id
Where order_status= 'delivered'

"""

result = pd.read_sql(query, conn)
print(result)

       OpC  count(order_status)  count(distinct customer_unique_id)
0  1.03342                96478                               93358


In [15]:
# this cell confirms the number of delivered orders

query="""
select count(order_status)
from orders
Where order_status = 'delivered'
"""

result = pd.read_sql(query, conn)
print(result)

   count(order_status)
0                96478


In [ ]:
#q7 top 10 customers

query= """
SELECT customer_unique_id,
Sum(payment_value) as 'total spend'
From customers
Join orders on
Customers.customer_id=orders.customer_id
Join payments on
Orders.order_id=payments.order_id
Where order_status= 'delivered'
Group by customer_unique_id
Order by sum(payment_value) desc

Limit 10


"""

result = pd.read_sql(query, conn)
print(result)

                 customer_unique_id  total spend
0  0a0a92112bd4c708ca5fde585afaa872     13664.08
1  da122df9eeddfedc1dc1f5349a1a690c      7571.63
2  763c8b1c9c68a0229c42c9fc6f662b93      7274.88
3  dc4802a71eae9be1dd28f5d788ceb526      6929.31
4  459bef486812aa25204be022145caa62      6922.21
5  ff4159b92c40ebe40454e3e6a7c35ed6      6726.66
6  4007669dec559734d6f53e029e360987      6081.54
7  eebb5dda148d3893cdaf5b5ca3040ccb      4764.34
8  48e1ac109decbb87765a3eade6854098      4681.78
9  c8460e4251689ba205045f3ea17884a1      4655.91


In [ ]:
#Q8 top ten cities by customer size


query= """
SELECT customer_city,
count(customer_city) as' # of customers'
From customers
Group by customer_city
Order by count(customer_city) desc
Limit 10
"""

result = pd.read_sql(query, conn)
print(result)

           customer_city   # of customers
0              sao paulo            15540
1         rio de janeiro             6882
2         belo horizonte             2773
3               brasilia             2131
4               curitiba             1521
5               campinas             1444
6           porto alegre             1379
7               salvador             1245
8              guarulhos             1189
9  sao bernardo do campo              938


In [ ]:
#q9 what product types make the most revenue

query= """

Select product_category_name, sum(payment_value) as revenue
FROM order_items
Join products
On Order_items.product_id=products.product_id
Join payments
On order_items.order_id=payments.order_id

Group by product_category_name
Order by sum(payment_value) desc
Limit 15;
"""

result = pd.read_sql(query, conn)
print(result)

     product_category_name     revenue
0          cama_mesa_banho  1712553.67
1             beleza_saude  1657373.12
2   informatica_acessorios  1585330.45
3         moveis_decoracao  1430176.39
4       relogios_presentes  1429216.68
5            esporte_lazer  1392127.56
6    utilidades_domesticas  1094758.13
7               automotivo   852294.33
8       ferramentas_jardim   838280.75
9               cool_stuff   779698.00
10       moveis_escritorio   646826.49
11              brinquedos   619037.69
12                   bebes   539845.66
13              perfumaria   506738.66
14               telefonia   486882.05


In [19]:
#q10 top ten items ordered, Products are anonymous in this data

query= """

Select product_id, count(product_id) as 'total'
FROM order_items
Group by product_id
Order by count(product_id) desc
Limit 10
"""

result = pd.read_sql(query, conn)
print(result)

                         product_id  total
0  aca2eb7d00ea1a7b8ebd4e68314663af    527
1  99a4788cb24856965c36a24e339b6058    488
2  422879e10f46682990de24d770e7f83d    484
3  389d119b48cf3043d311335e499d9c6b    392
4  368c6c730842d78016ad823897a372db    388
5  53759a2ecddad2bb87a079a1f1519f73    373
6  d1c427060a0f73f6b889a5c7c61f2ac4    343
7  53b36df67ebb7c41585e8d54d6772e08    323
8  154e7e31ebfa092203795c972e5804a6    281
9  3dd2a17168ec895c781a9191c1e95ad7    274


In [33]:
#Q11: Which Products have the highest average ratings?

query= """
SELECT avg(review_score) as average_rating, product_id
from Reviews
join order_items on
reviews.order_id=order_items.order_id
group by product_id
HAVING COUNT(*) > 14
order by avg(review_score) desc
limit 10
"""
result = pd.read_sql(query, conn)
print(result)

   average_rating                        product_id
0        5.000000  37eb69aca8718e843d897aa7b82f462d
1        4.958333  3e4176d545618ed02f382a3057de32b4
2        4.937500  62c89abe1afe3a23c17765d462718a4c
3        4.888889  de480e3ae31eea2d2d97d694c43172fc
4        4.869565  e7f85e7f0203b7b95cc1b4c21b4b070c
5        4.842105  0a4f9f421af66d2ea061fbb8883419f7
6        4.833333  73326828aa5efe1ba096223de496f596
7        4.833333  574597aaf385996112490308e37399ce
8        4.820513  f7f59e6186e10983a061ac7bdb3494d6
9        4.814815  990d135e28e075648cb7d83198fdccf4


In [40]:
#Q12: Most popular payment methods
query = """
SELECT payment_type, count(payment_type) as 'total'
from payments
group by payment_type
order by count(payment_type) desc
"""
result = pd.read_sql(query, conn)
print(result)

  payment_type  total
0  credit_card  76795
1       boleto  19784
2      voucher   5775
3   debit_card   1529
4  not_defined      3


In [45]:
#Q13: Average number of payment installments

query = """
Select  avg(payment_installments) as 'Avg installments'
from payments
"""
result = pd.read_sql(query, conn)
print(result)


   Avg installments
0          2.853349


In [58]:
#Q14: Average delivery time
query = """
SELECT
AVG(julianday(order_delivered_customer_date) - julianday(order_purchase_timestamp)) AS avg_delivery_time
from orders
where order_status = 'delivered'
"""
result = pd.read_sql(query, conn)
print(result)

   avg_delivery_time
0          12.558217


In [57]:
 #Q15 Relationship between payment installments and order value

query = """
SELECT payment_installments, AVG(payment_value)
FROM payments
GROUP BY payment_installments
ORDER BY payment_installments
"""
result = pd.read_sql(query, conn)
print(result)

    payment_installments  AVG(payment_value)
0                      0           94.315000
1                      1          112.420229
2                      2          127.228150
3                      3          142.539317
4                      4          163.976840
5                      5          183.465222
6                      6          209.849952
7                      7          187.673672
8                      8          307.737427
9                      9          203.440870
10                    10          415.085837
11                    11          124.932174
12                    12          321.678496
13                    13          150.462500
14                    14          167.962667
15                    15          445.553108
16                    16          292.694000
17                    17          174.602500
18                    18          486.483333
19                    20          615.801765
20                    21          243.700000
21        